## Установим нужные либы

In [5]:
!pip install findspark==2.0.1
!pip install pandas==2.0.0
!pip install numpy==1.24.2

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Using cached botocore-1.34.69-py3-none-any.whl (12.0 MB)
  Attempting uninstall: botocore
    Found existing installation: botocore 1.19.63
    Uninstalling botocore-1.19.63:
      Successfully uninstalled botocore-1.19.63
ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

boto3 1.16.7 requires botocore<1.20.0,>=1.19.7, but you'll have botocore 1.34.69 which is 

### Доустановим либы

In [ ]:
!pip install fsspec
!pip install s3fs
!pip install boto3

In [2]:
import os
import findspark

findspark.init()
findspark.find()

'/usr/lib/spark'

### Читанем первые несколько строк

In [20]:
# Read the first n lines of the file
n = 10  # Specify the number of lines to read


s3_filepath = "s3a://otus-task-n2/2019-08-22.txt"
first_n_lines_df = spark.read.text(s3_filepath).limit(n)

# Iterate over each row and print the content
for row in first_n_lines_df.collect():
    print(row['value'])

# tranaction_id | tx_datetime | customer_id | terminal_id | tx_amount | tx_time_seconds | tx_time_days | tx_fraud | tx_fraud_scenario
0,2019-08-22 06:51:03,0,711,70.91,24663,0,0,0
1,2019-08-22 05:10:37,0,0,90.55,18637,0,0,0
2,2019-08-22 19:05:33,0,753,35.38,68733,0,0,0
3,2019-08-22 07:21:33,0,0,80.41,26493,0,0,0
4,2019-08-22 09:06:17,1,981,102.83,32777,0,0,0
5,2019-08-22 18:41:25,3,205,34.20,67285,0,0,0
6,2019-08-22 03:12:21,3,0,47.20,11541,0,0,0
7,2019-08-22 22:36:40,6,809,139.39,81400,0,0,0
8,2019-08-22 17:23:29,7,184,87.24,62609,0,0,0


In [34]:
cols = ['# tranaction_id ',
 ' tx_datetime ',
 ' customer_id ',
 ' terminal_id ',
 ' tx_amount ',
 ' tx_time_seconds ',
 ' tx_time_days ',
 ' tx_fraud ',
 ' tx_fraud_scenario']

### Прочитаем данные в Data Frame

In [35]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, TimestampType


spark = SparkSession.builder \
    .appName("Data Cleaning") \
    .getOrCreate()

s3_filepath = "s3a://otus-task-n2/2019-08-22.txt"


columns = cols

df = spark.read. \
        option("sep", ","). \
        option("comment", "#"). \
        csv(s3_filepath, header=False).toDF(*cols)

df.show(150)

+----------------+-------------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
|# tranaction_id |       tx_datetime | customer_id | terminal_id | tx_amount | tx_time_seconds | tx_time_days | tx_fraud | tx_fraud_scenario|
+----------------+-------------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
|               0|2019-08-22 06:51:03|            0|          711|      70.91|            24663|             0|         0|                 0|
|               1|2019-08-22 05:10:37|            0|            0|      90.55|            18637|             0|         0|                 0|
|               2|2019-08-22 19:05:33|            0|          753|      35.38|            68733|             0|         0|                 0|
|               3|2019-08-22 07:21:33|            0|            0|      80.41|            26493|             0|         0|                 0|
|     

In [36]:
df = df.dropDuplicates()

In [37]:
df.show(10)

+----------------+-------------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
|# tranaction_id |       tx_datetime | customer_id | terminal_id | tx_amount | tx_time_seconds | tx_time_days | tx_fraud | tx_fraud_scenario|
+----------------+-------------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
|         7024049|2019-08-26 15:46:14|       484618|          807|      89.41|           402374|             4|         1|                 2|
|         7024135|2019-08-26 06:44:52|       484676|          696|      46.47|           369892|             4|         0|                 0|
|         7024180|2019-08-26 14:44:01|       484702|          261|      47.10|           398641|             4|         0|                 0|
|         7024300|2019-08-26 23:11:36|       484783|          882|     130.27|           429096|             4|         0|                 0|
|     

### Подсчитаем количество строк в DF

In [40]:
# Get the number of rows in the DataFrame
num_rows = df.count()

# Print the number of r`ows
print("Number of rows in the DataFrame:", num_rows)

Number of rows in the DataFrame: 46988418


### Взглянем на схему данных, которую по дефолту прочитало из csv. Видно, что в DF есть четкие типы данных по столбцам, попробуем к ним привести и взглянем на соответствие им

In [15]:
df.printSchema()

root
 |-- # tranaction_id : string (nullable = true)
 |--  tx_datetime : string (nullable = true)
 |--  customer_id : string (nullable = true)
 |--  terminal_id : string (nullable = true)
 |--  tx_amount : string (nullable = true)
 |--  tx_time_seconds : string (nullable = true)
 |--  tx_time_days : string (nullable = true)
 |--  tx_fraud : string (nullable = true)
 |--  tx_fraud_scenario: string (nullable = true)



### Посмотрим для каждого столбца есть ли данные, которые не конвертятся в заданный тип

In [31]:
# Look at rows that don't convert properly
def check_column_dtype(df, col):
    print(f"Checking column {col}")
    
    if col in ['# tranaction_id ',
                 ' customer_id ',
                 ' terminal_id ',
                 ' tx_time_seconds ',
                 ' tx_time_days ',
                 ' tx_fraud ',
                 ' tx_fraud_scenario']:
        
        df.filter(F.col(col).cast("int").isNull()).show(truncate=False)
    elif col == ' tx_amount ':
        df.filter(F.col(col).cast("double").isNull()).show(truncate=False)
    elif col == ' tx_datetime ':
        printdf.filter(F.to_timestamp(F.col(col)).isNull()).show(truncate=False)
    else:
        print("No such column")
        
for col in cols:
    check_column_dtype(df, col)

Checking column # tranaction_id 
+----------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
|# tranaction_id | tx_datetime | customer_id | terminal_id | tx_amount | tx_time_seconds | tx_time_days | tx_fraud | tx_fraud_scenario|
+----------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
+----------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+

None
Checking column  tx_datetime 
+----------------+-------------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
|# tranaction_id | tx_datetime       | customer_id | terminal_id | tx_amount | tx_time_seconds | tx_time_days | tx_fraud | tx_fraud_scenario|
+----------------+-------------------+-------------+-------------+-----------+-----------------+-------

### Похоже что отображение полночи как 24:00:00 не совсем корректно для дефолтного типо дейттайм, так что поправим его


### Также видим, что в файле с транзакциями за определенную дату есть транзакции и за другие даты, их бы отфильтровать

In [45]:
from pyspark.sql.functions import *

df = df.withColumn(" tx_datetime ", when(df[" tx_datetime "].substr(12, 8) == "24:00:00", 
                                       concat(date_add(df[" tx_datetime "], 1), lit(" 00:00:00"))
                                      ).otherwise(df[" tx_datetime "]))

### Проверим столбец еще раз

In [42]:
col = " tx_datetime "
check_column_dtype(df, col)

Checking column  tx_datetime 
+----------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
|# tranaction_id | tx_datetime | customer_id | terminal_id | tx_amount | tx_time_seconds | tx_time_days | tx_fraud | tx_fraud_scenario|
+----------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
+----------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+

None


### Приведем все столбцы в нужные типы

In [46]:
# Cast columns to desired data types
df = df.withColumn("# tranaction_id ", df["# tranaction_id "].cast("int"))
df = df.withColumn(" tx_datetime ", F.to_timestamp(df[" tx_datetime "]))
df = df.withColumn(" customer_id ", df[" customer_id "].cast("int"))
df = df.withColumn(" terminal_id ", df[" terminal_id "].cast("int"))
df = df.withColumn(" tx_amount ", df[" tx_amount "].cast("double"))
df = df.withColumn(" tx_time_seconds ", df[" tx_time_seconds "].cast("int"))
df = df.withColumn(" tx_time_days ", df[" tx_time_days "].cast("int"))`
df = df.withColumn(" tx_fraud ", df[" tx_fraud "].cast("int"))
df = df.withColumn(" tx_fraud_scenario", df[" tx_fraud_scenario"].cast("int"))

In [47]:
df.printSchema()

root
 |-- # tranaction_id : integer (nullable = true)
 |--  tx_datetime : timestamp (nullable = true)
 |--  customer_id : integer (nullable = true)
 |--  terminal_id : integer (nullable = true)
 |--  tx_amount : double (nullable = true)
 |--  tx_time_seconds : integer (nullable = true)
 |--  tx_time_days : integer (nullable = true)
 |--  tx_fraud : integer (nullable = true)
 |--  tx_fraud_scenario: integer (nullable = true)



### Проверим есть ли в столбцах отрицательные значения, что было бы некорректно

In [58]:
for col in cols:
    df.filter(F.col(col) < 0).show()

+----------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
|# tranaction_id | tx_datetime | customer_id | terminal_id | tx_amount | tx_time_seconds | tx_time_days | tx_fraud | tx_fraud_scenario|
+----------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
+----------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+

+----------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
|# tranaction_id | tx_datetime | customer_id | terminal_id | tx_amount | tx_time_seconds | tx_time_days | tx_fraud | tx_fraud_scenario|
+----------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
+----------------+-------------+-------------+-

### Видим, что есть

### Проверим дубликаты строк

In [55]:
# Identify duplicate rows based on all columns
duplicate_rows = df.groupBy(df.columns).count().filter(col("count") > 1)

# Show duplicate rows
duplicate_rows.show()

+----------------+-------------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+-----+
|# tranaction_id |       tx_datetime | customer_id | terminal_id | tx_amount | tx_time_seconds | tx_time_days | tx_fraud | tx_fraud_scenario|count|
+----------------+-------------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+-----+
|         4116196|2019-08-24 07:10:33|       628134|          678|     116.53|           198633|             2|         0|                 0|    2|
|        10416343|2019-08-28 16:45:18|       651437|          767|      20.01|           578718|             6|         0|                 0|    2|
|        20765589|2019-09-04 15:05:43|       259978|          604|      13.12|          1177543|            13|         0|                 0|    2|
|        46848100|2019-09-20 01:28:22|       910438|          509|       5.15|          2510902|            29| 

### Они тоже есть

In [66]:
spark.stop()

## Таким образом нашли и нужно почистить
1. Закомменченные строки
2. Некорректные даты (полночь)
3. Лишние даты (не соответствуют дню)
4. Отрицательные значения (в этом файле айдишники)
5. Дубликаты

### Напишем скрипт для фильтрации данных

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, DoubleType, StringType, TimestampType
from pyspark.sql.functions import *
import s3fs

import logging


cols = ['tranaction_id',
 'tx_datetime',
 'customer_id',
 'terminal_id',
 'tx_amount',
 'tx_time_seconds',
 'tx_time_days',
 'tx_fraud',
 'tx_fraud_scenario']


def list_bucket(bucket_name):
    fs = s3fs.S3FileSystem(anon=True,
                      endpoint_url="https://storage.yandexcloud.net")
    bucket_objects = fs.ls(bucket_name)
    bucket_objects = [x for x in bucket_objects if x.endswith(".txt")]
    
    return bucket_objects
                      
def replace_midnight(df):
    return df.withColumn("tx_datetime", when(df["tx_datetime"].substr(12, 8) == "24:00:00", 
                                       concat(date_add(df["tx_datetime"], 1), lit(" 00:00:00"))
                                      ).otherwise(df["tx_datetime"]))

def cast_dtypes(df):
    df = df.withColumn("tranaction_id", df["tranaction_id"].cast("int"))
    df = df.withColumn("tx_datetime", to_timestamp(df["tx_datetime"]))
    df = df.withColumn("customer_id", df["customer_id"].cast("int"))
    df = df.withColumn("terminal_id", df["terminal_id"].cast("int"))
    df = df.withColumn("tx_amount", df["tx_amount"].cast("double"))
    df = df.withColumn("tx_time_seconds", df["tx_time_seconds"].cast("int"))
    df = df.withColumn("tx_time_days", df["tx_time_days"].cast("int"))
    df = df.withColumn("tx_fraud", df["tx_fraud"].cast("int"))
    df = df.withColumn("tx_fraud_scenario", df["tx_fraud_scenario"].cast("int"))                  
    df = df.dropna()
    
    return df
                      
def drop_negative_values(df):
    for col_name in df.columns:
        if df.schema[col_name].dataType in ["int", "double"]:
            df = df.filter(col(col_name) >= 0)
    
    return df

def drop_dates(filepath, df):

    filename_date = filepath.split('/')[1].split('.')[0]              
    df = df.filter(to_date(col("tx_datetime")) == filename_date)
    return df

def save_to_parquet(output_bucket, filepath, df):
    
    fn = filepath.split("/")[1]
    output_path = f"s3a://{output_bucket}/{fn}"
    df.write.parquet(output_path)
    
    logging.info(f"File {fn} was successefully saved after cleaning")


if __name__ == "__main__":
    spark = SparkSession.builder \
        .appName("Data Cleaning") \
        .getOrCreate()

    
    bucket_name = "otus-task-n2"
    output_bucket = "otus-task-n3"
    file_paths = list_bucket(bucket_name)
    
    for filepath in file_paths:

        df = spark.read. \
                option("sep", ","). \
                option("comment", "#"). \
                csv(f"s3a://{filepath}", header=False).toDF(*cols)
                      
        
        df = df.dropDuplicates()
        df = drop_dates(filepath, df)              
        df = replace_midnight(df)
        df = cast_dtypes(df)
        df = drop_negative_values(df)
        df = df.orderBy("tranaction_id")
        
        save_to_parquet(output_bucket, filepath, df)
        

In [40]:
df.show(10)

+----------------+-------------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
|# tranaction_id |       tx_datetime | customer_id | terminal_id | tx_amount | tx_time_seconds | tx_time_days | tx_fraud | tx_fraud_scenario|
+----------------+-------------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
|               0|2019-08-22 06:51:03|            0|          711|      70.91|            24663|             0|         0|                 0|
|               1|2019-08-22 05:10:37|            0|            0|      90.55|            18637|             0|         0|                 0|
|               2|2019-08-22 19:05:33|            0|          753|      35.38|            68733|             0|         0|                 0|
|               3|2019-08-22 07:21:33|            0|            0|      80.41|            26493|             0|         0|                 0|
|     

In [42]:
df.filter(col(" customer_id ") < 0).show()

+----------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
|# tranaction_id | tx_datetime | customer_id | terminal_id | tx_amount | tx_time_seconds | tx_time_days | tx_fraud | tx_fraud_scenario|
+----------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
+----------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+



In [43]:
df.filter(to_date(col(" tx_datetime ")) == "2019-08-22").show()

+----------------+-------------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
|# tranaction_id |       tx_datetime | customer_id | terminal_id | tx_amount | tx_time_seconds | tx_time_days | tx_fraud | tx_fraud_scenario|
+----------------+-------------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
|               0|2019-08-22 06:51:03|            0|          711|      70.91|            24663|             0|         0|                 0|
|               1|2019-08-22 05:10:37|            0|            0|      90.55|            18637|             0|         0|                 0|
|               2|2019-08-22 19:05:33|            0|          753|      35.38|            68733|             0|         0|                 0|
|               3|2019-08-22 07:21:33|            0|            0|      80.41|            26493|             0|         0|                 0|
|     

In [44]:
df.filter(to_date(col(" tx_datetime ")) == "2019-09-06").show()

+----------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
|# tranaction_id | tx_datetime | customer_id | terminal_id | tx_amount | tx_time_seconds | tx_time_days | tx_fraud | tx_fraud_scenario|
+----------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+
+----------------+-------------+-------------+-------------+-----------+-----------------+--------------+----------+------------------+



In [58]:
fn = filepath.split("/")[1]
output_path = f"s3a://otus-task-n3/{fn}"
# Write DataFrame to Parquet format in the specified S3 bucket
df.write.parquet(output_path)

In [55]:
spark.stop()